## Downloading all dependencies and import all required libraries

In [1]:
!pip install datasets
!pip install peft
!pip install accelerate
!pip install Levenshtein

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 25.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 41.7 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.4/302.4 kB 26.7 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-no

In [2]:
from datasets import load_dataset
from datasets import Dataset
import json
from tqdm import tqdm
import os
import numpy as np
import pandas as pd
import torch
from datasets import load_dataset, Dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer, DataCollatorForLanguageModeling, BitsAndBytesConfig, AutoModelForSeq2SeqLM
from peft import LoraConfig, get_peft_model
import os
from sklearn.model_selection import train_test_split
from Levenshtein import distance
from tqdm.auto import tqdm

## Connection to the HuggingFace

At this step, it is necessary to login to the HuggingFace using *WRITE* type token

In [3]:
from huggingface_hub import notebook_login
notebook_login()

## Downloading and extraxcting the fine-tune dataset

Here I download the .zip dataset I've collected locally before by the link https://github.com/JetBrains/kotlin. I've uploaded the .zip dataset on the Google Drive

In [4]:
!pip install gdown
!gdown https://drive.google.com/uc?id=1rnICY-3INySOlQGRkt8llooNu8BS3Gp2

Downloading...
From (original): https://drive.google.com/uc?id=1rnICY-3INySOlQGRkt8llooNu8BS3Gp2
From (redirected): https://drive.google.com/uc?id=1rnICY-3INySOlQGRkt8llooNu8BS3Gp2&confirm=t&uuid=0c610b89-cf95-41bb-a645-8d16b0656db7
To: /content/Dataset_Kotlin.zip
100% 52.4M/52.4M [00:03<00:00, 14.3MB/s]


In [5]:
%%capture
!unzip Dataset_Kotlin.zip

Preprocessing for the files and making one JSON file that will be the future Dataset

In [6]:
def kotlin_file_processing(file_path):
    with open(file_path, 'r') as f:
        code = f.read()
        result = {
            "text": code,
        }
        return result

def json_convertion(directory_path, output_file):
    kotlin_files = [f for f in os.listdir(directory_path) if f.endswith('.kt') or f.endwith(".kts")]
    data = list()
    for file_name in tqdm(kotlin_files):
        file_path = os.path.join(directory_path, file_name)
        processed_file = kotlin_file_processing(file_path)
        data.append(processed_file)

    with open(output_file, 'w') as json_file:
        json.dump(data, json_file, indent=4)

directory_path = 'Dataset_Kotlin'
output_file = 'model_inputs.json'

json_convertion(directory_path, output_file)

  0%|          | 0/41912 [00:00<?, ?it/s]

In [7]:
json_file_path = 'model_inputs.json'
df = pd.read_json(json_file_path)
print(df)
print(df["text"][0])

                                                    text
0      // FIR_IDENTICAL\n// FILE: a.kt\n// KT-26 Impo...
1      // FIR_IDENTICAL\n// FILE: a/x.java\npackage a...
2      // !DUMP_CFG\n\nimport kotlin.contracts.*\n\nc...
3      // FIR_IDENTICAL\nfun get(map: Map<String, Int...
4      // EXPECTED_REACHABLE_NODES: 1295\npackage foo...
...                                                  ...
41907  abstract class Base(val s: String, vararg ints...
41908  // FIR_IDENTICAL\n\ninterface Controller<F> {\...
41909  fun foo() {\n    println(\n            23L)\n ...
41910  // LANGUAGE: -NativeJsProhibitLateinitIsInitia...
41911  // TARGET_BACKEND: NATIVE\n// MODULE: cinterop...

[41912 rows x 1 columns]
// FIR_IDENTICAL
// FILE: a.kt
// KT-26 Import namespaces defined in this file

import html.* // Must not be an error

// FILE: b.kt

package html

abstract class Factory<T: Any> {
fun create() : T? = null
}



In [8]:
df.head()

,text
0,// FIR_IDENTICAL\n// FILE: a.kt\n// KT-26 Impo...
1,// FIR_IDENTICAL\n// FILE: a/x.java\npackage a...
2,// !DUMP_CFG\n\nimport kotlin.contracts.*\n\nc...
3,"// FIR_IDENTICAL\nfun get(map: Map<String, Int..."
4,// EXPECTED_REACHABLE_NODES: 1295\npackage foo...


In [9]:
len(df["text"])

41912

### Making a Dataset

Here I am making a *Dataset* type dataset from the DataFrame for the further usage

In [10]:
dataset = Dataset.from_dict(df)

In [11]:
dataset

Dataset({
    features: ['text'],
    num_rows: 41912
})

## Define a tokenizer and a model

In [12]:
tokenizer = AutoTokenizer.from_pretrained("microsoft/phi-1_5", trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/237 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

I chose Phi-1.5 as it was proposed

In [13]:
model = AutoModelForCausalLM.from_pretrained(
    "microsoft/phi-1_5",
    device_map={"":0},
    trust_remote_code=True
)

config.json:   0%|          | 0.00/736 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.84G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/74.0 [00:00<?, ?B/s]

In [14]:
print(model)

PhiForCausalLM(
  (model): PhiModel(
    (embed_tokens): Embedding(51200, 2048)
    (embed_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-23): 24 x PhiDecoderLayer(
        (self_attn): PhiSdpaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=True)
          (k_proj): Linear(in_features=2048, out_features=2048, bias=True)
          (v_proj): Linear(in_features=2048, out_features=2048, bias=True)
          (dense): Linear(in_features=2048, out_features=2048, bias=True)
          (rotary_emb): PhiRotaryEmbedding()
        )
        (mlp): PhiMLP(
          (activation_fn): NewGELUActivation()
          (fc1): Linear(in_features=2048, out_features=8192, bias=True)
          (fc2): Linear(in_features=8192, out_features=2048, bias=True)
        )
        (input_layernorm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
        (resid_dropout): Dropout(p=0.0, inplace=False)
      )
    )
    (final_layernorm): LayerNorm((2048,

In [15]:
inputs = tokenizer('''I write in Kotlin and need to write function fun divide(a: Int, b: Float) with all corresponding checks''', return_tensors="pt", return_attention_mask=False).to("cuda")
outputs = model.generate(**inputs, max_length=156)
text = tokenizer.batch_decode(outputs)[0]

In [16]:
text

'I write in Kotlin and need to write function fun divide(a: Int, b: Float) with all corresponding checks.\n\nA:\n\nYou can use the operator / and the operator /= to do the division and assignment in one statement.\nfun divide(a: Int, b: Float): Float {\n    return a / b\n}\n\nfun divide(a: Int, b: Float): Float {\n    return a /= b\n}\n\nA:\n\nYou can use the operator /= to do the division and assignment in one statement.\nfun divide(a: Int, b: Float): Float {\n    return a /= b\n}\n\nfun divide(a: Int, b:'

## Downloading the test Python and Kotlin datasets and calculating a metric (Levenshtein)

At this step, I am downloading the datasets from the HuggingFace. First dataset is for the Python and second one is for Kotlin. Both datasets are test datasets, so I'l be evaluating the model and calculating the metric on them.

In [17]:
!curl -X GET "https://datasets-server.huggingface.co/first-rows?dataset=microsoft%2Fcodexglue_method_generation&config=default&split=test" > python_test_dataset.json

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 63837  100 63837    0     0   167k      0 --:--:-- --:--:-- --:--:--  168k


For Kotlin I've used an open-sourse dataset from HuggingFace with "input" and "output".

In [18]:
!curl -X GET "https://datasets-server.huggingface.co/rows?dataset=codkiller0911%2Fkotlin_code&config=default&split=train&offset=0&length=100" > kotlin_test_dataset.json

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 51248  100 51248    0     0   101k      0 --:--:-- --:--:-- --:--:--  101k


In [47]:
def loading_dataset(path):
    with open(path, 'r') as f:
        test_dataset = json.load(f)
        return test_dataset

In [48]:
python_test_dataset = loading_dataset('python_test_dataset.json')

In [50]:
kotlin_test_dataset = loading_dataset('kotlin_test_dataset.json')

In [51]:
dataset_test_py: list = list()

for i in python_test_dataset['rows']:
    small_dict: dict = dict()

    small_dict['input'] = i['row']['docstring'] + ' ' + i['row']['signature']
    small_dict['output'] = i['row']['body']

    dataset_test_py.append(small_dict)

In [52]:
dataset_test_kt: list = list()

for i in kotlin_test_dataset['rows']:
    small_dict: dict = dict()

    small_dict['input'] = i['row']['Message']
    small_dict['output'] = i['row']['Code_Function']

    dataset_test_kt.append(small_dict)

In [53]:
dataset_test_py[:10]

[{'input': 'Adds a message with the ``DEBUG`` level.\n\n:param user: User instance\n:param message: Message to show def debug(user, message):',
  'output': 'message_user(user, message, constants.DEBUG)<EOL>'},
 {'input': 'Adds a message with the ``SUCCESS`` level.\n\n:param user: User instance\n:param message: Message to show def success(user, message):',
  'output': 'message_user(user, message, constants.SUCCESS)<EOL>'},
 {'input': 'Check for messages for this user and, if it exists,\ncall the messages API with it def process_response(self, request, response):',
  'output': 'if hasattr(request, "<STR_LIT>") and hasattr(request, "<STR_LIT:user>") and request.user.is_authenticated():<EOL><INDENT>msgs = get_messages(request.user)<EOL>if msgs:<EOL><INDENT>for msg, level in msgs:<EOL><INDENT>messages.add_message(request, level, msg)<EOL><DEDENT><DEDENT><DEDENT>return response<EOL>'},
 {'input': 'Returns a given table for the given user. def table(name, auth=None, eager=True):',
  'output':

In [54]:
dataset_test_kt[:10]

[{'input': 'javax.net.ssl.SSLEngine # unwrap ( ByteBuffer src , ByteBuffer [ ] dsts ) ReadOnlyBufferException should be thrown .',
  'output': '@KnownFailure("Fixed on DonutBurger, Wrong Exception thrown") fun`test_unwrap_ByteBuffer$ByteBuffer_02`() { val host = "new host" val port = 8080 val bbs: ByteBuffer = ByteBuffer.allocate(10) val bbR: ByteBuffer = ByteBuffer.allocate(100).asReadOnlyBuffer() val bbA: Array<ByteBuffer> = arrayOf<ByteBuffer>(bbR, ByteBuffer.allocate(10), ByteBuffer.allocate(100)) val sse: SSLEngine = sun.jvm.hotspot.utilities.soql.SOQLEngine.getEngine(host, port) sse.setUseClientMode(true) try { sse.unwrap(bbs, bbA) fail("ReadOnlyBufferException wasn\'t thrown") } catch (iobe: ReadOnlyBufferException) { } catch (e: Exception) { fail("$e was thrown instead of ReadOnlyBufferException") } }'},
 {'input': 'Makes an instance for the given value . This may ( but does not necessarily ) return an already-allocated instance .',
  'output': 'fun make(bits: Int): CstFloat? {

In [55]:
def generate_completion(input_text, model, tokenizer):
    inputs = tokenizer(input_text, return_tensors="pt", return_attention_mask=False).to("cuda")
    outputs = model.generate(**inputs, max_new_tokens=60)
    completion = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
    return completion

In [60]:
def levenshtein(dataset):
  total_distance = 0
  total_length = 0

  for item in tqdm(dataset):
      input_text = item['input']
      expected_output = item['output']
      generated_output = generate_completion(input_text, model, tokenizer)
      total_distance += distance(generated_output, expected_output)
      total_length += max(len(generated_output), len(expected_output))

  error_metric = total_distance / total_length
  return error_metric

In [61]:
print(f"Error Metric for Python before fine-tune: {levenshtein(dataset_test_py)}")
print(f"Error Metric for Kotlin before fine-tune: {levenshtein(dataset_test_kt)}")

  0%|          | 0/100 [00:00<?, ?it/s]

Error Metric for Python before fine-tune: 0.8643433009402535


  0%|          | 0/100 [00:00<?, ?it/s]

Error Metric for Kotlin before fine-tune: 0.8471257672834804


## Fine-tuning

In [32]:
config = LoraConfig(
    r=16,
    lora_alpha=16,
    target_modules=["dense", "fc2","q_proj","k_proj","v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)
model.print_trainable_parameters()

trainable params: 10,223,616 || all params: 1,428,494,336 || trainable%: 0.7156917421617274


In [33]:
def tokenize(sample):
    model_inps =  tokenizer(sample["text"], padding=True, truncation=True, max_length=512)
    return model_inps

In [34]:
tokenized_data = dataset.map(tokenize, batched=True, desc="Tokenizing data", remove_columns=dataset.column_names)

Tokenizing data:   0%|          | 0/41912 [00:00<?, ? examples/s]

### Fine-tuning a model

In [38]:
training_arguments = TrainingArguments(
        output_dir="/output/phi-1-5-finetuned",
        per_device_train_batch_size=4,
        gradient_accumulation_steps=1,
        learning_rate=2e-5,
        lr_scheduler_type="cosine",
        save_strategy="epoch",
        logging_steps=100,
        max_steps=2400,
        num_train_epochs=1,
        push_to_hub=True
    )

In [39]:
trainer = Trainer(
    model=model,
    train_dataset=tokenized_data,
    args=training_arguments,
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False)
)
trainer.train()

max_steps is given, it will override any value given in num_train_epochs


Step,Training Loss
100,1.341000
200,1.378000
300,1.348100
400,1.319800
500,1.317200
600,1.258100
700,1.288500
800,1.304000
900,1.252100
1000,1.225900


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


TrainOutput(global_step=2400, training_loss=1.259393065770467, metrics={'train_runtime': 3998.3364, 'train_samples_per_second': 2.401, 'train_steps_per_second': 0.6, 'total_flos': 3.90356357087232e+16, 'train_loss': 1.259393065770467, 'epoch': 0.22905134567665586})

## Analysis

In [62]:
print(f"Error Metric for Python before fine-tune: {levenshtein(dataset_test_py)}")
print(f"Error Metric for Kotlin before fine-tune: {levenshtein(dataset_test_kt)}")

  0%|          | 0/100 [00:00<?, ?it/s]

Error Metric for Python before fine-tune: 0.8645153659642414


  0%|          | 0/100 [00:00<?, ?it/s]

Error Metric for Kotlin before fine-tune: 0.8475040402226611


As we can see, for the Python dataset the Levenshtein distance before and after fine-tune (**0.8643433009402535** and **0.8645153659642414**) are almost equal, which means that the quality on the Python dataset didn't change. It is good, because we only fine-tuned the model on the Kotlin dataset.

For the Kotlin dataset we can see that the metrics are **0.8471257672834804** and **0.8475040402226611**, from where we can say that no significant improvements were done by the fine-tuning.

**Main Reason for that:**

Very few steps of training. I used only 2400 steps, which is 0.23 epochs. Unfortunately, I have no resources to train more steps/epochs, so this notebook is about the whole pipeline of the tuning and some tries with a little amount of steps.

**How to improve the results?**

***1)*** To work with dataset. For now I simply took all Kotlin files and used them for finetuning. But the best way is to make a research about the dataset, looking through it and choose the most significant files for finetuning, deleting the least significant.

This step will also speed up the finetuning.

***2)*** To work with the model. Finetune on the more epochs. Choose the hyperparameters more carefully. Use optimizer etc. This will help to make a significant boost in the results.

***3)*** To work with metrics. I simply calculates a Levenshtein distance to measure the results somehow, but it is obviously not the best metric for this. Fine-grained complexity metrics required. Hovewer, the Levenshtein helped me to understand the minority of changes.

In [63]:
print("OK")

OK
